In [ ]:
!pip uninstall torchtext torchaudio torchdata -y

Found existing installation: torchtext 0.17.1
Uninstalling torchtext-0.17.1:
  Successfully uninstalled torchtext-0.17.1
Found existing installation: torchaudio 2.2.1+cu121
Uninstalling torchaudio-2.2.1+cu121:
  Successfully uninstalled torchaudio-2.2.1+cu121
Found existing installation: torchdata 0.7.1
Uninstalling torchdata-0.7.1:
  Successfully uninstalled torchdata-0.7.1


In [ ]:
!python3 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113  --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 871.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 54.6 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.17.1+cu121
    Uninstalling torchvision-0.17.1+cu121:
      Successfully uninstalled torchvision-0.17.1+cu121


In [ ]:
!python3 -m pip install pillow==9.4.0 typing-inspect==0.9.0 importlib-metadata==6.8.0 importlib-resources==6.0.1 matplotlib==3.7.3 matplotlib-inline==0.1.6 omegaconf==2.3.0 einops==0.7.0 tqdm==4.66.1 pytorch-lightning==1.4.2 torchmetrics==0.6.0 bitsandbytes==0.41.1 pytorch_fid taming-transformers-rom1504

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 921.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 916.6/916.6 kB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=f7d25a264213768e202204f8226942c8177e18ee7bd1d1142901c53329c2cd72
  Stored in directory: /root/.cache/pip/whee

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/CV_Project/Large-Image-Diffusion'

/content/drive/.shortcut-targets-by-id/17qQqck-NGopMjdHyhi3bCDakrHtEqfYc/CV_Project/Large-Image-Diffusion


In [ ]:
%pwd

'/content/drive/.shortcut-targets-by-id/17qQqck-NGopMjdHyhi3bCDakrHtEqfYc/CV_Project/Large-Image-Diffusion'

In [ ]:
import torch
from pathlib import Path
from notebooks.utils import get_model, load_model_from_config
from ldm.util import instantiate_from_config
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import default_collate
from ldm.models.diffusion.plms import PLMSSampler
from torchvision.utils import make_grid
from einops import rearrange
from PIL import Image
import pickle
import os
from tqdm import tqdm

In [ ]:
device = torch.device("cuda:0")

In [ ]:
# load pretrained weights
model_path = Path("/content/drive/MyDrive/crc_ibot_20x/")
model, config = get_model(model_path, device, "last.ckpt")

Loading model from /content/drive/MyDrive/crc_ibot_20x/checkpoints/last.ckpt
No module 'xformers'. Proceeding without it.
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 400.92 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'vanilla' with 512 in_channels


In [ ]:
# sampling params
batch_size = 8
scale = 2

sampler = PLMSSampler(model)
ddim_steps = 50
shape = [3,64,64]

In [ ]:
# Define classes and data directories
class_names = ['ADI', 'BACK', 'DEB', 'LYM', 'MUC', 'MUS', 'NORM', 'STR', 'TUM']
base_path = Path("/content/drive/MyDrive/CV_Project/SSL_Embeddings")
output_path = Path("/content/drive/MyDrive/CV_Project/Synthetic_images")

# Ensure output directories exist
for class_name in class_names:
    os.makedirs(output_path / class_name, exist_ok=True)

# Processing loop
for class_name in tqdm(class_names, desc="Processing Classes"):
    for i in range(1, 6):  # Assuming 5 pickle files per class
        file_path = base_path/class_name/f"{class_name}_image_embeddings_batch_{i}.pkl"
        with open(file_path, "rb") as f:
            items = pickle.load(f)

        # Loop over all items in batches
        for start_idx in tqdm(range(0, len(items), batch_size), desc="Batches", leave=False):
          batch = default_collate(items[start_idx:start_idx + batch_size])

          with torch.no_grad(), model.ema_scope():
            batch["feat_patch"] = batch["feat_patch"].to(device)
            batch_uncond = {**batch}
            batch_uncond["feat_patch"] = torch.zeros_like(batch["feat_patch"])

            cc = model.get_learned_conditioning(batch)
            uc = model.get_learned_conditioning(batch_uncond)

            samples_ddim, _ = sampler.sample(
                S=ddim_steps,
                conditioning=cc,
                batch_size=len(batch['feat_patch']),
                shape=shape,
                verbose=False,
                unconditional_guidance_scale=scale,
                unconditional_conditioning=uc,
                )

            x_samples_ddim = model.decode_first_stage(samples_ddim)
            x_samples_ddim = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)
            x_samples_ddim = (x_samples_ddim * 255).to(torch.uint8).cpu()

            # Save images
            for j, image in enumerate(x_samples_ddim):
              img_path = output_path / class_name / f"{class_name}_{i}_{start_idx + j}.png"
              Image.fromarray(image.permute(1, 2, 0).numpy()).save(img_path)


Batches: 100%|██████████| 125/125 [42:05<00:00, 20.29s/it]
                                                          
Batches: 100%|██████████| 125/125 [41:56<00:00, 20.19s/it]
                                                          
Batches: 100%|██████████| 125/125 [42:07<00:00, 20.23s/it]
                                                          
Batches: 100%|██████████| 125/125 [42:13<00:00, 20.24s/it]
                                                          
Batches: 100%|██████████| 125/125 [41:58<00:00, 19.33s/it]
                                                          
Batches: 100%|██████████| 125/125 [39:59<00:00, 19.13s/it]
                                                          
Batches: 100%|██████████| 125/125 [39:51<00:00, 19.14s/it]
                                                          
Batches: 100%|██████████| 125/125 [39:47<00:00, 19.12s/it]
                                                          
Processing Classes: 100%|██████████| 2/2 [6:54:01<00:00,